# Tarefa 02

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

#### 1. Carregue os dados da tarefa anterior

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Definir os ativos escolhidos (Exemplo: Apple, Tesla, Microsoft)
ativos = ["AAPL", "TSLA", "MSFT"]

# Definir o período de 2 anos atrás até hoje
data_fim = datetime.today().strftime('%Y-%m-%d')
data_inicio = (datetime.today() - timedelta(days=2*365)).strftime('%Y-%m-%d')

# Baixar os dados do Yahoo Finance
dados_ativos = {ativo: yf.download(ativo, start=data_inicio, end=data_fim) for ativo in ativos}

# Salvar os dados em arquivos CSV
for ativo, df in dados_ativos.items():
    df.to_csv(f"{ativo}.csv")

print("Dados baixados e salvos em arquivos CSV.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Dados baixados e salvos em arquivos CSV.


#### 2. Média móvel

1. Faça uma média móvel  em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu
2. Utilizando a fução ```rolling()```, calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
2. Monte um gráfico de linhas para as médias móveis dos ativos da sua base
4. Monte um gráfico de linhas para os desvios padrão em janela móvel
5. Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

In [5]:
for ativo, df in dados_ativos.items():
    df["Media_Movel_90d"] = df[("Close", ativo)].rolling(window=90).mean()
for ativo, df in dados_ativos.items():
    df.columns = [col[0] for col in df.columns]  # Mantém apenas o primeiro nível ('Close', 'High', etc.)
    df["Media_Movel_90d"] = df["Close"].rolling(window=90).mean()


In [7]:
import plotly.express as px

for ativo, df in dados_ativos.items():
    # Removendo valores NaN antes de plotar
    df_filtrado = df.dropna(subset=["Close", "Media_Movel_90d"])
    
    fig = px.line(df_filtrado, x=df_filtrado.index, y=["Close", "Media_Movel_90d"], 
                  title=f"Média Móvel de 90 dias - {ativo}",
                  labels={"value": "Preço Ajustado", "variable": "Legenda"})
    fig.show()


In [9]:
for ativo, df in dados_ativos.items():
    if "Desvio_Padrao_90d" not in df.columns:
        print(f"Erro: 'Desvio_Padrao_90d' não encontrada em {ativo}. Recalculando...")
        df["Desvio_Padrao_90d"] = df["Close"].rolling(window=90).std()

    # Filtrar NaN para evitar erros ao plotar
    df_filtrado = df.dropna(subset=["Desvio_Padrao_90d"])
    
    fig = px.line(df_filtrado, x=df_filtrado.index, y="Desvio_Padrao_90d", 
                  title=f"Desvio Padrão (Volatilidade) - {ativo}",
                  labels={"Desvio_Padrao_90d": "Desvio Padrão"})
    fig.show()


Erro: 'Desvio_Padrao_90d' não encontrada em AAPL. Recalculando...


Erro: 'Desvio_Padrao_90d' não encontrada em TSLA. Recalculando...


Erro: 'Desvio_Padrao_90d' não encontrada em MSFT. Recalculando...


#### 3. Gráfico de *candlestick*
1. Selecione o período dos últimos 60 dias (corridos) para um dos ativos da sua base
2. Monte um gráfico de *candlestick* para esses dados.

In [10]:
import plotly.graph_objects as go

# Escolher um ativo para o gráfico (exemplo: AAPL)
ativo = "AAPL"

# Filtrar os últimos 60 dias
df_candlestick = dados_ativos[ativo].iloc[-60:]

# Criar o gráfico Candlestick
fig = go.Figure(data=[go.Candlestick(
    x=df_candlestick.index,
    open=df_candlestick["Open"],
    high=df_candlestick["High"],
    low=df_candlestick["Low"],
    close=df_candlestick["Close"],
    name=ativo
)])

fig.update_layout(
    title=f"Gráfico Candlestick - Últimos 60 dias ({ativo})",
    xaxis_title="Data",
    yaxis_title="Preço",
    xaxis_rangeslider_visible=False
)

fig.show()
